In [3]:
import populator

In [6]:
data_dir = 'data'
# Second item in tuple is max batch size for requests.
exchanges = [('binance', 1000)]
pairs = ['TUSD/USDT']
tick_size = '1h'
start = '2000-01-01T00:00:00Z'

In [7]:
populator.populate(data_dir, exchanges, pairs, tick_size, start)

Batch Size: 1000
Total Entries: 168334
Note: Entries will not exist if they predate exchange history.
Entries Processed: 162427
Entries Processed: 163427
Entries Processed: 164427
Entries Processed: 165437
Entries Processed: 166437
Entries Processed: 167437
Entries Processed: 168334
Scraping for data/binance/TUSD-USDT-1h-2000-01-01T00-00-00Z-all.csv succeeded.


In [8]:
df = populator.load_data_as_frame(data_dir, 'binance', 'TUSD/USDT', tick_size, start)

In [31]:
import backtrader as bt

In [32]:
class StableCoinStrategy(bt.Strategy):
    def next(self):
        if not self.position:
            self.buy(size=self.data.close[0] / (self.data.close[0] + 1.0) * 10000.0)
        elif self.data.close[0] > 1.0:
            self.sell(size=1000)
        else:
            self.buy(size=1000)

In [35]:
cerebro = bt.Cerebro()
cerebro.addstrategy(StableCoinStrategy)
data = bt.feeds.PandasData(dataname=df)
cerebro.adddata(data)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.00, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

starting_cash = 10**5
cerebro.broker.setcash(starting_cash)
cerebro.broker.setcommission(commission=0.001)
sim = cerebro.run()

In [36]:
print('Final Value: {}'.format(cerebro.broker.getvalue()))
print('PNL %: {}'.format(100.0 * (cerebro.broker.getvalue() / starting_cash) - 100.0))
print('Sharpe Ratio: {}'.format(sim[0].analyzers.sharpe.get_analysis()['sharperatio']))
print('Max Drawdown: {}'.format(sim[0].analyzers.drawdown.get_analysis().max.drawdown))

Final Value: 124692.52743611578
PNL %: 24.692527436115782
Sharpe Ratio: 0.5967983101938887
Max Drawdown: 214.4826026548319
